In [1]:
import os
os.chdir('../')
os.getcwd()

'd:\\Code Workspace\\Personal Project\\Personal Project - End to End Loan Status Classification'

In [2]:
from dotenv import load_dotenv
load_dotenv();

In [3]:
# os.environ['MLFLOW_TRACKING_URI']

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: str
    test_data_path: str
    model_path: str
    all_params: dict
    metric_file_path: Path
    target_column: str
    mlflow_uri: str

In [5]:
from MLProject.constants import *
from MLProject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.SVC
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_path=config.metric_file_path,
            target_column=schema.name,
            mlflow_uri = os.environ['MLFLOW_TRACKING_URI']
        )

        return model_evaluation_config

In [6]:
import os
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
from MLProject.utils.common import load_bin, save_json

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = load_bin(self.config.model_path)

        X_test = test_data.drop([self.config.target_column], axis= 1)
        y_test = test_data[[self.config.target_column]]
        
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predicted = model.predict(X_test)

            (accuracy, f1, precision, recall) = self._eval_metrics(y_test, predicted)

            ## Saving metrics as ocal
            scores = {
                "accuracy": accuracy,
                "f1": f1,
                "precision": precision,
                "recall": recall
            }
            save_json(path = Path(self.config.metric_file_path), data = scores)

            ## Log params
            mlflow.log_params(self.config.all_params)

            ## Log metrics
            mlflow.log_metric("accuracy", accuracy)
            mlflow.log_metric("f1", f1)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)

            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case.
                # docs: https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="SVC")
            else:
                mlflow.sklearn.log_model(model, "model")

    @staticmethod
    def _eval_metrics(actual, pred):
        accuracy = accuracy_score(actual, pred)
        f1 = f1_score(actual, pred)
        precision = precision_score(actual, pred)
        recall = recall_score(actual, pred)
        return (accuracy, f1, precision, recall)

In [7]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config= model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

| 2024-01-23 22:37:55,553 | INFO | common | yaml file: config\config.yaml loaded successfully |
| 2024-01-23 22:37:55,556 | INFO | common | yaml file: params.yaml loaded successfully |
| 2024-01-23 22:37:55,560 | INFO | common | yaml file: schema.yaml loaded successfully |
| 2024-01-23 22:37:55,564 | INFO | common | created directory at: artifacts |
| 2024-01-23 22:37:55,565 | INFO | common | created directory at: artifacts\model_evaluation |
| 2024-01-23 22:37:55,709 | INFO | common | binary file loaded from: artifacts\model_trainer\model.joblib |
| 2024-01-23 22:39:04,533 | INFO | common | json file saved at: artifacts\model_evaluation\metrics.json |


Registered model 'SVC' already exists. Creating a new version of this model...
2024/01/23 22:39:17 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: SVC, version 2
Created version '2' of model 'SVC'.
